# 1. Imports

In [ ]:
from __future__ import annotations

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import os
import re
import gc
from tqdm.notebook import tqdm

from typing import List, Union, Any
import string

import warnings
warnings.filterwarnings("ignore")

# 2. Data Extraction

## 2.1 Creating Empty list for csv filenames
This csv files will be extracted into one `pandas.DataFrame`

In [ ]:
csv_collection: List[str] = []

Extracting csv filenames into already created empty list `csv_collection`

In [ ]:
for dir_name, _, file_names in os.walk('kaggle/input/ukraine-russian-crisis-twitter-dataset-1-2-m-rows/UkraineWar/UkraineWar/'):
    for filename in file_names:
        fullpath = os.path.join(dir_name, filename)
        csv_collection.append(fullpath)

print(',\n'.join(csv_collection[:5]))

## 2.2 Loading dataset with significant events

Due to inconsistent tweets dataset filenames format we forced to use for loop to filter `csv_collection` 

In [ ]:
def extract(string: str, pattern: str):
    return re.findall(pattern, string)

## Create and fill dataframe


We created types dict to avoid some warnings while storing data to dataframe. It is also increases loading speed.

In [ ]:
types = {
    'userid': np.uint, 
    'username': object, 
    'acctdesc': object, 
    'location': object, 
    'following': np.uintc, 
    'followers': np.uintc,
    'totaltweets': np.uint, 
    'tweetid': np.uint, 
    'retweetcount': np.ushort, 
    'text': object, 
    'hashtags': object, 
    'language': object, 
    'coordinates': object,
    'favorite_count': np.uintc, 
    'is_retweet': object, 
    'original_tweet_id': np.uint,
    'original_tweet_userid': np.uint, 
    'original_tweet_username': object,
    'in_reply_to_status_id': np.uint, 
    'in_reply_to_user_id': np.uint,
    'in_reply_to_screen_name': object, 
    'is_quote_status': object, 
    'quoted_status_id': np.uint,
    'quoted_status_userid': np.uint, 
    'quoted_status_username': object
}

# Important Note
The dataset contains literally a lot of data (really a lot, because we waited more than 20 minutes to store all data from significant dates (approximately 60 dates). In these days was approximately 10.5M tweets.

In the latest version of the notebook our team decided to view only tweets posted on the 1st of March, 2022 - the day of Kharkiv government building airstrike, day before and after it.

**UPDATE** 
In the 4th version of the notebook the author of the dataset has suggested us to take into account difference in time (all times in the dataset are in UTC+0), so we will extend the time for spectating on data.

In [ ]:
mar01_tweets = [csv for csv in csv_collection if "MAR02" in csv or "MAR01" in csv or "FEB28" in csv]
all_tweets = pd.read_csv(mar01_tweets.pop(), compression='gzip', dtype=types)
for csv in mar01_tweets:
    all_tweets = pd.concat([all_tweets, pd.read_csv(csv, compression='gzip', dtype=types)])
gc.collect()

## Important Note
If you want ot store a full version of the dataset into your Jupyter Notebook, please, uncomment the cell below. 

**Don't forget to replace `mar01_tweets` variable with `filtered_csv_collection` on previous cell.**

In [ ]:
# for csv in tqdm(filtered_csv_collection, desc='Files: '):
#     temp = pd.read_csv(csv, compression='gzip', dtype=types)
#     all_tweets = pd.concat([all_tweets, temp])
#     del temp
#     gc.collect()

# 3. Data review

In [ ]:
all_tweets.info()

In [ ]:
all_tweets.isna().sum()

The dataset contains Not a Number (NaN) values. 


## TODO:
- [x] set index to `userid`
- [x] remove `username`
- [x] remove account description (`acctdesc`)
- [x] remove `usercreatedts`
- [x] parse `hashtags`
- [x] remove `coordinates`
- [x] remove `extractedts`
- [x] parse `location`
- [ ] transform time into Kyiv timezone (UTC+3 in winter and UTC+2 in summer)
- [x] extend dataset for 2 days as we need to look at tweets before and after missile attack

## Clearing data from dataset

In [ ]:
all_tweets = all_tweets.set_index('userid')
all_tweets = all_tweets.drop(columns=['username', 'acctdesc', 'usercreatedts', 'coordinates', 'extractedts'])

## Parsing hashtags

Print an example of hashtags

In [ ]:
all_tweets['hashtags'].values[0]

In [ ]:
all_tweets['hashtags'] = all_tweets['hashtags'].apply(lambda string: extract(string, "'text': \'(\S*)'"))

## Parsing location

## TODO:
- [x] remove text translation because we found dataset with alternate city and country names.

In [ ]:
all_tweets['location'].values[:100]

In [ ]:
print(f"Dataset size before empty locations deletion: {len(all_tweets)}")
all_tweets = all_tweets[all_tweets['location'].notna()]
print(f"Dataset size after empty locations deletion: {len(all_tweets)}")
print(f"Unique locations size: {all_tweets['location'].unique().shape}")

## Cleaning tweets

In [ ]:
def clean_text(tweet: str) -> list[str | Any]:
    raw_string: str = r'{}'.format(tweet)
    demojified: str = re.sub(r"\\u\S{4}", '', raw_string)
    text: str = "".join([letter for letter in demojified if letter not in string.punctuation])
    return list(filter(lambda word: word != '', re.split(r'\W+', text)))

In [ ]:
all_tweets['text'] = all_tweets['text'].apply(lambda tweet: clean_text(tweet))

In [ ]:
datacities = pd.read_csv('kaggle/input/geonames-all-cities-with-a-population/geonames-all-cities-with-a-population-1000.csv', sep=';')
dicttowns = dict(zip(datacities['Name'], datacities['Country name EN']))
countries = set(datacities['Country name EN'])

In [ ]:
datacities = datacities[['Geoname ID', 'Name', 'ASCII Name', 'Alternate Names', 'Country Code', 'Country name EN']]

In [ ]:
datacities.head()

In [ ]:
import json
with open('kaggle/input/locations.json') as json_file:
    loctranslator: dict = json.load(json_file)
all_tweets['location'] = all_tweets['location'].map(loctranslator).fillna('Other')
all_tweets.drop(all_tweets[all_tweets['location'] == 'Other'].index, inplace = True)

## Plotting

In [ ]:
n_first = 20

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot(x=all_tweets['language'].value_counts()[:n_first].index, y=all_tweets['language'].value_counts()[:n_first])
plt.xticks(rotation=90)
plt.yscale("log")
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.pie(x=all_tweets['language'].value_counts()[:n_first], labels=all_tweets['language'].value_counts()[:n_first].index)
plt.show()

In [ ]:
plt.figure(figsize=(15, 20))
sns.barplot(y=all_tweets['location'].value_counts()[:n_first].index, x=all_tweets['location'].value_counts()[:n_first])
plt.xscale("log")
plt.show()

We removed records with `Other` locations because there is no appropriate information even though they make up the bulk of the dataset.

In [ ]:
other_location_tweets = all_tweets[all_tweets['location'] == 'Other']
all_tweets = all_tweets.drop(all_tweets[all_tweets['location'] == 'Other'].index)

In [ ]:
all_tweets["tweetcreatedts"] = pd.to_datetime(all_tweets["tweetcreatedts"])
print(all_tweets["tweetcreatedts"])

Let's see 

In [ ]:
grouped_by_time = all_tweets[["tweetcreatedts", "text"]].groupby(pd.Grouper(key="tweetcreatedts",freq='H')).size().reset_index()

plt.figure(figsize=(20, 15))
sns.barplot(data=grouped_by_time, y="tweetcreatedts", x=0)
plt.show()

Study the anomaly peak on plot between 09:00:00 UTC the 1st of March and the 04:00:00 UTC the 2nd of March.

In [ ]:
# TODO. implement study

## Curator advices:
- [ ] Consider 2 sides of the conflict - Ukraine and Russia (means analyze reactions from both sides)
- [ ] Look at countries-satelites reaction (both for Ukraine and Russia)
- [ ] Take into account that there are refugees, which can distort some data (for example, Ukrainian refugees in Poland are still Ukrainians, but can change their account information, because Twitter doesn't verify all user info)
- [ ] Analyze reaction of people in G20 countries and other aliances.
    Aliances to take into account:
    - G20
    - complete it later :)

### Group tweets by language

In [ ]:
datasets_by_language = {}
for language in all_tweets['language'].unique():
    datasets_by_language[language] = all_tweets[all_tweets['language'] == language]


### Extract Ukrainian and Russian tweets from dataset
As Ukrainians are bilinguals (detailed study [The History of Bilingualism in Ukraine and Its Role in the Present Day Political Crisis](https://www.cairn-int.info/article-E_CSP_017_0203--the-history-of-bilingualism-in-ukraine.htm)) we firstly extracted tweets by language and then separate it by tweeters locations.

In [ ]:
ukrainian_tweets = datasets_by_language['uk']
russian_tweets = datasets_by_language['ru']

In [ ]:
plt.figure(figsize=(20, 10))
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].pie(ukrainian_tweets['location'].value_counts()[:n_first], labels=ukrainian_tweets['location'].value_counts().index[:n_first])
ax[0].set_title('Ukrainian tweets locations')
ax[1].pie(russian_tweets['location'].value_counts()[:n_first], labels=russian_tweets['location'].value_counts().index[:n_first])
ax[1].set_title('Russian tweets locations')
plt.show()

In [ ]:
g20_dict: dict[str, list[str]] = {
    'g1': ['Australia', 'Canada', 'Saudi Arabia', 'United States'],
    'g2': ['India', 'Russia', 'South Africa', 'Turkey'],
    'g3': ['Argentina', 'Brazil', 'Mexico'],
    'g4': ['France', 'Germany', 'Italy', 'United Kingdom']
}
g20_list: list[str] = [
    'Argentina', 'Australia', 'Brazil', 'Canada',
    'China', 'France', 'Germany', 'India',
    'Indonesia', 'Italy', 'South Korea', 'Japan',
    'Mexico', 'Russia', 'Saudi Arabia', 'South Africa',
    'Turkey', 'the United Kingdom', 'United States', 'European Union'
]